In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 9, 6
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
#%run autoencoder.py 64 2 0.25 --uneven --model_type gru --sigma 1e-9 --sim_type autoencoder/even/noise0 --embedding 8
%run survey_lcs.py 64 1 0.25

[]
/Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/survey_lcs/gru_064_x1_1m03_drop25
Loading /Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/survey_lcs/gru_064_x1_1m03_drop25/weights.h5...


In [ ]:
[t_i for t_i, m_i, e_i in X_asas_list]

In [ ]:
from keras.utils.np_utils import to_categorical
from cesium.datasets import fetch_asas_training

data = fetch_asas_training()
X_asas_list = [np.c_[data['times'][i], data['measurements'][i],
                data['errors'][i]] for i in range(len(data['times']))]
#n_max = max(len(t) for t in data['times'])
n_max = X.shape[1]
X_asas = pad_sequences(X_asas_list, maxlen=n_max, value=-1., dtype='float')
classnames, indices = np.unique(data['classes'], return_inverse=True)
Y_asas = to_categorical(indices, len(classnames))

In [ ]:
%%time
encode = K.function([model.layers[0].input, model.layers[args.num_layers + 1].input], [model.layers[2].output])
encoding = encode([X[:5], X[:5, :, [0, 2]]])[0]
#encoding_asas = encode([X_asas, X_asas[:, :, [0, 2]]])[0]

In [ ]:
i = -1

In [ ]:
np.median(X[0, :, 1])

In [ ]:
i += 1
# TODO if args.even: ...
inds = (X[i, :, -1] >= 0.)
t = X[i, inds, 0]
m = X[i, inds, 1]
e = X[i, inds, 2]
#plt.plot(t, m, 'o')
plt.errorbar(t, m, e, None, 'o')

pred_i = model.predict({'main_input': X[i:i+1], 'aux_input': X[i:i+1, :, [0, 2]]})
plt.plot(t, pred_i[0, inds, 0], 'o')

#w_r, A_r, phi_r, b_r = pred_gru[i]
#x_r = A_r * np.sin(2 * np.pi * w_r * t + phi_r) + b_r
#plt.plot(t, x_r, '--')

#plt.legend(['Noisy', 'Original'])

In [ ]:
np.corrcoef(encoding, Y[:encoding.shape[0]], rowvar=0)[:6, 6:]

In [ ]:
sns.jointplot(encoding[:, 5], Y[:encoding.shape[0], 0], kind='hex')

In [ ]:
sns.jointplot(encoding[:, 1], Y[:encoding.shape[0], 1], kind='hex')

In [ ]:
sns.jointplot(encoding[:, 5], Y[:encoding.shape[0], 2], kind='hex')

In [ ]:
sns.jointplot(encoding[:, 3], Y[:encoding.shape[0], 3], kind='hex')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR

model = RandomForestRegressor(n_estimators=128)
#model = ElasticNet()
#model = SVR(kernel='rbf')

model.fit(encoding, Y[:encoding.shape[0], 0])

In [ ]:
encoding_test = encode([X[test]])[0]
Y_pred = model.predict(encoding_test)

In [ ]:
#sns.jointplot(Y[test, 0], Y_pred[:, 0])
sns.jointplot(Y[test, 0], Y_pred)